# Links from feedparser are sent to newpaper3k to extract textual information such as summary,keywords,text.

In [ ]:
import pandas as pd
from pandas import DataFrame
import feedparser,io
import re
from newspaper import Article


from pandas import ExcelWriter
from pandas import ExcelFile
df = pd.read_excel('RSS_ExtractionFormatV2.xlsx',index_col = 0)

In [ ]:
import datetime

import time
import pytz # pip install pytz
import tzlocal # pip install tzlocal

In [ ]:
df.head(100)

In [ ]:
df2 = df[['Name','Rss','NTags','SCOPE','Type']]

In [ ]:
df2.head()

In [ ]:
feeds = []
posts =[]
description = []
counter  = 0
for url in df2['Rss']:
    
    
   
    
    
    feed = feedparser.parse(url)
    Name = df2['Name'][counter]
    Ntags = df2['NTags'][counter]
    Scope = df2['SCOPE'][counter]
    Type = df2['Type'][counter]
    counter = counter +1
    for post in feed.entries:
        
        try:
        
            post.author
        except Exception as e:
            posts.append((Name,Ntags,Scope,Type,post.title,post.link,post.published,Ntags))
            print(e)
            print("continuing.....")
            continue
        
       
        
       

        posts.append((Name,Ntags,Scope,Type,post.title,post.link,post.published,post.author))
        
    for post in feed.entries:  
        try:
            post.description
        except Exception as d:
            description.append("Not available")
            print(d)
            print("continuing")
            continue
        description.append(post.description)
            
        
        
        
df3 = pd.DataFrame(posts, columns=['Name','source','Scope','Tags','title', 'url','published','authors'])


In [ ]:
df3['meta-description'] = description

In [ ]:
# remove all tags(image tags ,src tags.etc)
count =0
for des in df3['meta-description']:
    
    clean = re.compile('<.*?>')
    
    df3['meta-description'][count] = re.sub(clean, '', df3['meta-description'][count])
    count = count +1

In [ ]:
df3.head(50)

In [ ]:
df3['publishedTime'] = pd.to_datetime(df3.published)
df3['publishedDate'] = df3['publishedTime'] 
df3['publish_dateX'] = df3['publishedTime']
df3['publish_date'] = df3['publishedTime']

df3.head()


In [ ]:
count = 0
for pub in df3['publishedDate']:
    #pub = pub.utcnow().date()
    df3['publish_date'][count] = df3['publishedDate'][count].strftime('%Y%m%d%H%M%S')
    df3['publish_dateX'][count] = df3['publishedDate'][count].strftime('%Y%m%d%H%M%S')
    
    df3['publishedTime'][count] = df3['publishedTime'][count].time()
    df3['publishedDate'][count] = df3['publishedDate'][count].date()
    #df3['timezone'][count] = df3['published'][count].strftime("%z")
    count = count +1

In [ ]:
df3.head(5)

In [ ]:
#int(df3['publish_date'][5])
count = 0
for pub in df3['publish_dateX']:
    df3['publish_dateX'][count] = int(df3['publish_dateX'][count])
    count = count +1

In [ ]:
df3.head()

In [ ]:
# stored run time from previous run is used to remove already parsed/extracted links
with open ("run_time.txt", "r") as myfile:
    previous_run=myfile.readline()
    if previous_run == "":
        previous_run = '00000000000000'
        
        

In [ ]:
previous_run = int(previous_run)
print(previous_run)

In [ ]:
count = 0
Dlist = []
for x in df3['publish_dateX']:
    if df3['publish_dateX'][count] <  previous_run:
        #print(df3['publish_dateX'][count])
        Dlist.append(count)
    count = count+1

In [ ]:
print(Dlist)

In [ ]:
df3 = df3.drop(df3.index[Dlist])

In [ ]:
#df3.head()


In [ ]:
#df3['publish_date'] = pd.to_datetime(df3.published)

In [ ]:
df3.head()

In [ ]:
df3 = df3.reset_index()
del df3['index']

In [ ]:
df3.head()

In [ ]:
timezone = datetime.datetime.now(pytz.timezone('Australia/Sydney')).strftime('%z')

In [ ]:
count = 0
for pub in df3['publish_date']:
    
    
    df3['publish_date'][count] = '' + df3['publish_date'][count] + timezone 
    df3['publish_date'][count] = df3['publish_date'][count].replace('+','')
    count = count +1

In [ ]:
df3.head(30)

In [ ]:
## replacing + fro timezone shouldnt yield any problems as rss feeds are focused on canberra and usually have same timezone as sydney. 
## in case of including world sources it might be better to convet time to local format and repalace +

In [ ]:
runtime = datetime.datetime.now(pytz.timezone('Australia/Sydney')).strftime('%Y%m%d%H%M%S')
#runtime = runtime.replace('+','')
print (runtime)

In [ ]:
# storing the run time- 
with open("run_time.txt", "w") as text_file:
    print(f"{runtime}", file=text_file)

In [ ]:
df3.shape

In [ ]:
from newspaper import Article

In [ ]:

Text= []
Summary = []
Keywords = []
for url in df3['url']:
    
    try:
        
        article = Article(url)
        article.download()
        article.html
        article.parse()
    except Exception as e:
        print(url)
        Text.append('error')
        Summary.append('error')
        Keywords.append('error')
        print(e)
        print("continuing.....")
        continue
    text = article.text
    article.nlp()
    keywords = article.keywords
    summary = article.summary
    Text.append(text)
    Summary.append(summary)
    Keywords.append(keywords)
    

    
    
    
   

In [ ]:
df3['cleaned_text'] = Text
df3['summary'] = Summary
df3['meta_keywords'] = Keywords

In [ ]:
df3['url'][100]

In [ ]:

#print(df3['link'][300])

In [ ]:
print(df3['title'][100])

In [ ]:
print(df3['cleaned_text'][100])

In [ ]:
print(df3['summary'][100])

In [ ]:
print(df3['meta-description'][100])

In [ ]:
df3.head(500)